In [0]:
%matplotlib inline
import numpy as np
import pandas as pd
import sklearn
import seaborn as sns 
import matplotlib.pyplot as plt
import re
import keras 

Using TensorFlow backend.


In [0]:
from keras.preprocessing.text import Tokenizer 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split 
from keras.models import Sequential 
from keras.preprocessing.sequence import pad_sequences 
from keras.layers import Dense, Flatten, LSTM, Embedding, SpatialDropout1D
from keras.utils.np_utils import to_categorical 

In [0]:
from google.colab import files

In [0]:
uploaded = files.upload()

Saving Sentiment.csv to Sentiment (1).csv


In [0]:
pd.set_option('display.max_columns',30)
df = pd.read_csv('Sentiment.csv')
df.head()

,id,candidate,candidate_confidence,relevant_yn,relevant_yn_confidence,sentiment,sentiment_confidence,subject_matter,subject_matter_confidence,candidate_gold,name,relevant_yn_gold,retweet_count,sentiment_gold,subject_matter_gold,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,1,No candidate mentioned,1.0,yes,1.0,Neutral,0.6578,None of the above,1.0000,NaN,I_Am_Kenzi,NaN,5,NaN,NaN,RT @NancyLeeGrahn: How did everyone feel about...,NaN,2015-08-07 09:54:46 -0700,629697200650592256,NaN,Quito
1,2,Scott Walker,1.0,yes,1.0,Positive,0.6333,None of the above,1.0000,NaN,PeacefulQuest,NaN,26,NaN,NaN,RT @ScottWalker: Didn't catch the full #GOPdeb...,NaN,2015-08-07 09:54:46 -0700,629697199560069120,NaN,NaN
2,3,No candidate mentioned,1.0,yes,1.0,Neutral,0.6629,None of the above,0.6629,NaN,PussssyCroook,NaN,27,NaN,NaN,RT @TJMShow: No mention of Tamir Rice and the ...,NaN,2015-08-07 09:54:46 -0700,629697199312482304,NaN,NaN
3,4,No candidate mentioned,1.0,yes,1.0,Positive,1.0000,None of the above,0.7039,NaN,MattFromTexas31,NaN,138,NaN,NaN,RT @RobGeorge: That Carly Fiorina is trending ...,NaN,2015-08-07 09:54:45 -0700,629697197118861312,Texas,Central Time (US & Canada)
4,5,Donald Trump,1.0,yes,1.0,Positive,0.7045,None of the above,1.0000,NaN,sharonDay5,NaN,156,NaN,NaN,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,NaN,2015-08-07 09:54:45 -0700,629697196967903232,NaN,Arizona


In [0]:
drop_list = []
print (df.shape)
for cols in df.columns:
  if ((df[cols].count()/13871.0)*100.0 < 25.0):
    drop_list.append(str(cols))
print (drop_list)
df = df.drop(drop_list , axis = 1)

# removing irrelevant columns 
df = df.drop(['tweet_id' , 'tweet_location','tweet_created' ,'candidate' ,'user_timezone' , 'name' ,'id' , 'subject_matter' , 'relevant_yn'] , axis = 1)
print ("\nnew shape")
print (df.shape)

(13871, 21)
['candidate_gold', 'relevant_yn_gold', 'sentiment_gold', 'subject_matter_gold', 'tweet_coord']

new shape
(13871, 7)


In [0]:
print (df.columns)
df.head()
df['sentiment'].value_counts()
df = df[df.sentiment != "Neutral"]
df['sentiment'] = df['sentiment'].map({'Negative': 0 , 'Positive' : 1})


Index(['candidate_confidence', 'relevant_yn_confidence', 'sentiment',
       'sentiment_confidence', 'subject_matter_confidence', 'retweet_count',
       'text'],
      dtype='object')


In [0]:
x = df[['candidate_confidence', 'relevant_yn_confidence',
       'sentiment', 'sentiment_confidence', 'subject_matter_confidence',
       'retweet_count','text']]
x.head()

,candidate_confidence,relevant_yn_confidence,sentiment,sentiment_confidence,subject_matter_confidence,retweet_count,text
1,1.0000,1.0,1,0.6333,1.0000,26,RT @ScottWalker: Didn't catch the full #GOPdeb...
3,1.0000,1.0,1,1.0000,0.7039,138,RT @RobGeorge: That Carly Fiorina is trending ...
4,1.0000,1.0,1,0.7045,1.0000,156,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...
5,0.6332,1.0,1,0.6332,1.0000,228,"RT @GregAbbott_TX: @TedCruz: ""On my first day ..."
6,1.0000,1.0,0,0.6761,1.0000,17,RT @warriorwoman91: I liked her and was happy ...


In [0]:
df_text = x[['text' , 'sentiment']]

In [0]:
df_text['text'] = df_text['text'].apply(lambda x : x.lower() )
# if the line doesn't start with a-z , A-Z , 0-9 or space then replace it with space 
df_text['text'] = df_text['text'].apply(lambda x : re.sub('[^a-zA-Z0-9\s]' , '', x))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
for idx , row in df_text.iterrows():
#   print (row[0])
  row[0] = row[0].replace('rt','')
#   print (row[0])
#   break
# preparing training dataset
max_words = 2000
tokens = Tokenizer(num_words = max_words , split = " ")
# count the instances or letters in the tweets
tokens.fit_on_texts(df_text['text'].values)
print (tokens)
x = tokens.texts_to_sequences(df_text['text'].values)
print (x)
# pad_sequence ensures that all seq have same length
x = pad_sequences(x)
print (x)


[[3, 359, 121, 1, 713, 2, 40, 59, 235, 37, 208, 7, 173, 1742, 13, 1303, 1387, 734], [3, 17, 282, 250, 6, 809, 103, 170, 27, 134, 7, 1, 171, 13, 2, 232, 714, 18], [3, 1244, 2, 308, 24, 1921, 1, 1608, 214, 13, 1, 693, 7, 184, 205, 367, 680], [3, 126, 18, 54, 261, 405, 10, 83, 301, 435, 1304, 1743, 1134, 63, 1922, 192, 2, 52], [3, 10, 1135, 170, 9, 22, 1305, 64, 10, 605, 186, 22, 187, 5, 35, 1, 556, 20, 810, 2, 45, 735], [13, 1, 3, 168, 97, 541, 35, 1, 104, 606, 53, 60, 1744, 8, 18, 607, 2], [3, 40, 147, 27, 811, 14, 2, 910], [24, 13, 47, 17], [3, 38, 1923, 48, 5, 1088, 21, 70, 2, 173, 542, 7, 1, 59, 274, 332], [32, 1188, 54, 446, 23, 55, 144, 1, 2, 22], [3, 661, 10, 520, 302, 166, 56, 13, 1306, 181, 14, 48, 2], [303, 368, 1, 7, 879, 15, 93, 458, 269, 6, 270, 2], [3, 265, 459, 951, 15, 446, 23, 108, 13, 18, 2, 468], [3, 608, 53, 185, 130, 174, 109, 30, 1, 2, 52, 276, 46, 7, 109], [759, 880, 6, 575, 13, 1307, 138, 995, 6, 1, 43, 150, 37, 1, 759, 1, 594, 427, 783, 15, 1, 50, 2], [3, 1388, 1

In [0]:
model = Sequential()
embed_dim = 128
lstm_out  = 196
# Turns positive integers (indexes) into dense vectors of fixed size
# This layer can only be used as the first layer in a model.
model.add(Embedding(max_words , embed_dim , input_length = x.shape[1]))
model.add(SpatialDropout1D(.4))
model.add(LSTM(lstm_out , dropout = .2 , recurrent_dropout = .2))
model.add(Dense(2 , activation = 'softmax'))


In [0]:
model.compile(loss = 'categorical_crossentropy' , optimizer = 'adam' , metrics = ['accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 28, 128)           256000    
_________________________________________________________________
spatial_dropout1d_3 (Spatial (None, 28, 128)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


[[3, 53, 79, 339, 449, 23, 2, 413, 360, 96, 30, 52, 1021, 1], [3, 349, 124, 1929, 2, 595, 1, 30, 52, 227, 36, 196, 6, 173, 1396, 11, 1555, 1328, 833], [3, 63, 491, 6, 9, 2, 1, 22, 1777, 11, 685, 512], [3, 18, 277, 236, 7, 724, 97, 164, 25, 131, 6, 2, 177, 11, 1, 212, 771, 17], [3, 1216, 1, 294, 24, 2, 1665, 234, 11, 2, 658, 6, 174, 186, 396, 596], [3, 117, 17, 48, 208, 330, 10, 72, 273, 414, 1329, 1930, 1172, 64, 1931, 190, 1, 57], [3, 10, 1217, 164, 9, 22, 1397, 65, 10, 597, 182, 22, 165, 5, 33, 2, 620, 21, 990, 1, 50, 809], [165, 17, 746, 397, 42, 621, 180, 1556, 1], [11, 2, 3, 167, 103, 529, 33, 2, 102, 622, 46, 58, 1778, 8, 17, 646, 1], [3, 30, 115, 25, 867, 15, 1, 901], [24, 11, 49, 18], [3, 1779, 40, 1932, 47, 5, 659, 20, 66, 1, 173, 598, 6, 2, 52, 158, 322], [35, 1125, 48, 450, 23, 53, 151, 2, 1, 22], [3, 3, 1126, 344, 290, 1, 1933], [3, 608, 10, 599, 314, 171, 56, 1934, 11, 1477, 181, 15, 47, 1], [336, 378, 2, 6, 1022, 14, 86, 465, 282, 7, 264, 1], [3, 248, 513, 922, 14, 450, 2

In [0]:
y = pd.get_dummies(df_text['sentiment']).values
x_train , x_test , y_train , y_test = train_test_split(x,y,test_size = .33)

In [0]:
batch_size = 32 
model.fit(x_train , y_train , epochs = 7 , batch_size = batch_size , verbose = 2)

Epoch 1/7
 - 24s - loss: 0.1675 - acc: 0.9304
Epoch 2/7
 - 25s - loss: 0.1516 - acc: 0.9354
Epoch 3/7
 - 25s - loss: 0.1434 - acc: 0.9416
Epoch 4/7
 - 25s - loss: 0.1341 - acc: 0.9459
Epoch 5/7
 - 25s - loss: 0.1207 - acc: 0.9494
Epoch 6/7
 - 25s - loss: 0.1229 - acc: 0.9513
Epoch 7/7
 - 25s - loss: 0.1110 - acc: 0.9528


In [0]:
# validatin the set 
validation_size = 1500
x_validate = x_test[-validation_size:]
y_validate = y_test[-validation_size:]
x_test = x_test[:-validation_size]
y_test = y_test[:-validation_size]

print (x_test)

[]


In [0]:
score , acc =  model.evaluate(x_test , y_test , verbose = 2 , batch_size = batch_size )
print (score)
print (acc)


ValueError: ignored